In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

#pyspark.sql.SparkSession builder from Session
spark=SparkSession.builder\
.master("local")\
.appName("DFLolo")\
.getOrCreate()
print(type(spark))


<class 'pyspark.sql.session.SparkSession'>


In [45]:
# Schema definition (not mandatory)
df_schema=None;
df_schema = [StructField('id', IntegerType(), True), 
             StructField('fname', StringType(), True), 
             StructField('lname', StringType(), True),
             StructField('age', IntegerType(), True)]
df_struct = StructType(fields=df_schema)

# DataFrame creation from CSV
df1=spark.read.schema(df_struct).csv(path='sample_data/cust1.csv', sep=',',header=True,inferSchema=False)
print("DF1")
df1.show()
df1.printSchema()

# DataFrame #2 creation from CSV
df2=spark.read.csv(path='sample_data/cust1.csv', sep=',',header=True,inferSchema=True)
print("DF2")
df2.show()
df2.printSchema()



DF1
+---+-------+--------+---+
| id|  fname|   lname|age|
+---+-------+--------+---+
|  1|      L|Leturgez| 42|
|  2|      V|Leturgez| 42|
|  3|      L|Leturgez|  8|
|  4|      O|Leturgez|  6|
+---+-------+--------+---+

root
 |-- id: integer (nullable = true)
 |-- fname: string (nullable = true)
 |-- lname: string (nullable = true)
 |-- age: integer (nullable = true)

DF2
+---+-------+--------+---+
| Id|  fname|   lname|age|
+---+-------+--------+---+
|  1|      L|Leturgez| 42|
|  2|      V|Leturgez| 42|
|  3|      L|Leturgez|  8|
|  4|      O|Leturgez|  6|
+---+-------+--------+---+

root
 |-- Id: integer (nullable = true)
 |-- fname: string (nullable = true)
 |-- lname: string (nullable = true)
 |-- age: integer (nullable = true)



In [57]:
df1.createOrReplaceTempView("tt")

# Filter
df2.where(df2['fname']=='      L').show()
spark.sql("SELECT id,lname,fname,age FROM tt WHERE fname='      L'").show()

df2.where(\
    (df2['fname']=='      L') |\
    (df2['fname']=='      V')\
).show()
spark.sql("SELECT id,lname,fname,age FROM tt WHERE fname='      L' OR fname='      V'").show()

df2.where(\
    (df2['age']>3) &\
    (df2['age']<=8)\
).show()
spark.sql("SELECT * FROM tt WHERE age>3 AND age<=8").show()


+---+-------+--------+---+
| Id|  fname|   lname|age|
+---+-------+--------+---+
|  1|      L|Leturgez| 42|
+---+-------+--------+---+

+---+--------+-------+---+
| id|   lname|  fname|age|
+---+--------+-------+---+
|  1|Leturgez|      L| 42|
+---+--------+-------+---+

+---+-------+--------+---+
| Id|  fname|   lname|age|
+---+-------+--------+---+
|  1|      L|Leturgez| 42|
|  2|      V|Leturgez| 42|
+---+-------+--------+---+

+---+--------+-------+---+
| id|   lname|  fname|age|
+---+--------+-------+---+
|  1|Leturgez|      L| 42|
|  2|Leturgez|      V| 42|
+---+--------+-------+---+

+---+-----+--------+---+
| Id|fname|   lname|age|
+---+-----+--------+---+
|  3|    L|Leturgez|  8|
|  4|    O|Leturgez|  6|
+---+-----+--------+---+

+---+-----+--------+---+
| id|fname|   lname|age|
+---+-----+--------+---+
|  3|    L|Leturgez|  8|
|  4|    O|Leturgez|  6|
+---+-----+--------+---+



In [58]:
df1.createOrReplaceTempView("tt")

df2.groupBy(df2['lname']).avg().show()

df2.groupBy(df2['lname']).agg({'age':'avg'}).show()
spark.sql("select lname,avg(age) from tt group by lname").show()




+--------+-------+--------+
|   lname|avg(Id)|avg(age)|
+--------+-------+--------+
|Leturgez|    2.5|    24.5|
+--------+-------+--------+

+--------+--------+
|   lname|avg(age)|
+--------+--------+
|Leturgez|    24.5|
+--------+--------+

+--------+--------+
|   lname|avg(age)|
+--------+--------+
|Leturgez|    24.5|
+--------+--------+



In [59]:
df1.createOrReplaceTempView("tt")

df2.orderBy(df2['age']).show()
spark.sql("select * from tt order by age").show()

df2.orderBy(df2['age'].desc()).show()
spark.sql("select * from tt order by age desc").show()

+---+-------+--------+---+
| Id|  fname|   lname|age|
+---+-------+--------+---+
|  4|      O|Leturgez|  6|
|  3|      L|Leturgez|  8|
|  1|      L|Leturgez| 42|
|  2|      V|Leturgez| 42|
+---+-------+--------+---+

+---+-------+--------+---+
| id|  fname|   lname|age|
+---+-------+--------+---+
|  4|      O|Leturgez|  6|
|  3|      L|Leturgez|  8|
|  1|      L|Leturgez| 42|
|  2|      V|Leturgez| 42|
+---+-------+--------+---+

+---+-------+--------+---+
| Id|  fname|   lname|age|
+---+-------+--------+---+
|  1|      L|Leturgez| 42|
|  2|      V|Leturgez| 42|
|  3|      L|Leturgez|  8|
|  4|      O|Leturgez|  6|
+---+-------+--------+---+

+---+-------+--------+---+
| id|  fname|   lname|age|
+---+-------+--------+---+
|  1|      L|Leturgez| 42|
|  2|      V|Leturgez| 42|
|  3|      L|Leturgez|  8|
|  4|      O|Leturgez|  6|
+---+-------+--------+---+

